In [1]:
import pandas as pd
import numpy as np
from Code.UtilityFunctions.wikidata_functions import wikidata_query, category_query, min_qid, get_all_wikidata_claims, compare_qids, categories_dict_singular, get_qid_label
from Code.UtilityFunctions.get_data_path import get_path

In [3]:
location_mappings = pd.read_csv(get_path('location_mappings_expanded.csv'))

In [13]:
location_mappings = location_mappings.dropna(subset=['city']).sort_values(by=['cityLabel'])

In [15]:

location_mappings.loc[location_mappings['cityLabel'].str.startswith('Q'), 'cityLabel'].unique()

array(['Q110916412', 'Q12530186', 'Q12536050', 'Q12564962', 'Q12564965',
       'Q20485552', 'Q20485660', 'Q31455252', 'Q31455253', 'Q31455254',
       'Quail Ridge', 'Quakertown'], dtype=object)